In [1]:
import numpy as np
from scipy.io import loadmat
import scipy.optimize as spo

In [2]:
data=loadmat('ex8_movies.mat')

In [3]:
data

{'R': array([[1, 1, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'Y': array([[5, 4, 0, ..., 5, 0, 0],
        [3, 0, 0, ..., 0, 0, 5],
        [4, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Dec  1 17:19:26 2011',
 '__version__': '1.0'}

In [4]:
r=data['R']
y=data['Y']
r.shape,y.shape

((1682, 943), (1682, 943))

In [5]:
float(y[0].sum())/r[0].sum()    #average rating for 1st movie

3.8783185840707963

In [6]:
nm,nu=y.shape
nm,nu

(1682, 943)

In [7]:
def cost(params,y,r,n):
    X=np.reshape(params[:nm*n],(nm,n))
    theta=np.reshape(params[nm*n:],(nu,n))
    err=(X.dot(theta.T)-y)*r
    J=np.sum(err**2.0)/2.0
    
    dJX=err.dot(theta)
    dJtheta=err.T.dot(X)
    
    dJ=np.concatenate((dJX.ravel(),dJtheta.ravel()))
    return J,dJ

In [8]:
#to check we have been provided pre-trained params
param_data=loadmat('ex8_movieParams.mat')
param_data

{'Theta': array([[ 0.28544362, -1.68426509,  0.26293877, ...,  0.76723235,
         -1.10460164, -0.25186708],
        [ 0.50501321, -0.45464846,  0.31746244, ...,  1.09306336,
         -1.20029436, -0.39161676],
        [-0.43191656, -0.47880449,  0.84671111, ...,  1.36333976,
         -0.32523542, -0.19468212],
        ..., 
        [ 1.0586926 , -0.8087176 ,  0.56935771, ...,  0.80421422,
         -0.74346778, -0.11985885],
        [ 0.89058932, -0.12894734,  0.35560466, ...,  1.37426807,
         -0.7526549 , -0.81135311],
        [ 0.82414136, -0.38892594,  1.06940502, ...,  1.28423703,
         -0.98349993, -0.53184838]]),
 'X': array([[ 1.0486855 , -0.40023196,  1.19411945, ...,  0.861721  ,
         -0.69728994,  0.28874563],
        [ 0.78085123, -0.38562591,  0.52119779, ...,  0.70402073,
         -0.48583521, -0.56462407],
        [ 0.64150886, -0.54785385, -0.08379638, ...,  0.83854643,
         -0.69483208, -1.13479631],
        ..., 
        [ 0.21952237, -0.20047886,  0.

In [9]:
theta=param_data['Theta']
X=param_data['X']
theta.shape,X.shape

((943, 10), (1682, 10))

In [10]:
params=np.concatenate((X.ravel(),theta.ravel()))

In [11]:
params.shape

(26250,)

In [12]:
J,dJ=cost(params,y,r,10)
J,dJ

(27918.64012454421,
 array([-6.26184144,  2.45936046, -6.87560329, ..., -6.56073746,
         5.20459188,  2.65003952]))

In [13]:
dJ.shape

(26250,)

In [14]:
def cost_reg(params,y,r,n,l):
    X=np.reshape(params[:nm*n],(nm,n))
    theta=np.reshape(params[nm*n:],(nu,n))
    err=(X.dot(theta.T)-y)*r
    J=np.sum(err**2.0)/2.0+l*(np.sum(theta**2.0)+np.sum(X**2.0))/2
    
    #dJX=err.dot(theta)+l*X
    #dJtheta=err.T.dot(X)+l*theta
    
    #dJ=np.concatenate((dJX.ravel(),dJtheta.ravel()))
    return J
def grad(params,y,r,n,l):
    X=np.reshape(params[:nm*n],(nm,n))
    theta=np.reshape(params[nm*n:],(nu,n))
    err=(X.dot(theta.T)-y)*r
    dJX=err.dot(theta)+l*X
    dJtheta=err.T.dot(X)+l*theta
    dJ=np.concatenate((dJX.ravel(),dJtheta.ravel()))
    return dJ

In [ ]:
lr=10
org_params=spo.fmin_bfgs(f=cost_reg,x0=params,fprime=grad,args=(y,r,10,lr),maxiter=100)

In [36]:
org_params

     fun: 71873.41373852719
     jac: array([  5.05106913e-06,  -3.61509146e-06,   3.76336585e-06, ...,
        -3.37739020e-06,  -9.77246000e-07,  -9.93675749e-07])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 3621
     nit: 2399
    njev: 3620
  status: 2
 success: False
       x: array([ 1.03574345, -0.60272956,  1.07026546, ...,  0.71281032,
       -1.53391643, -0.34906298])